In [1]:
!ls 

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!cat income.train.txt.5k | cut -f 10 -d ',' | sort | uniq -c

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
1251 / (1251 + 3749)

0.2502

In [4]:
!cat income.dev.txt | cut -f 10 -d ',' | sort | uniq -c

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
236 / (236 + 764)

0.236

In [6]:
!cat income.train.txt.5k | cut -f 1 -d ',' | sort -nk1| head -1

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat income.train.txt.5k | cut -f 1 -d ',' | sort -nk1| tail -1

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
!cat income.train.txt.5k | cut -f 8 -d ',' | sort -nk1| head -1

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
!cat income.train.txt.5k | cut -f 8 -d ',' | sort -nk1| tail -1

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
!cat income.train.txt.5k | cut -f 2 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!cat income.train.txt.5k | cut -f 3 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!cat income.train.txt.5k | cut -f 4 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
!cat income.train.txt.5k | cut -f 5 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
!cat income.train.txt.5k | cut -f 6 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
!cat income.train.txt.5k | cut -f 7 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
!cat income.train.txt.5k | cut -f 9 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
!cat income.train.txt.5k | cut -f 1 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
!cat income.train.txt.5k | cut -f 8 -d ',' | sort | uniq | wc -l

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
import numpy as np

In [20]:
import time

In [21]:
def binarization(file, mapping=None):
    # Intialize data array from file
    data = map(lambda s: s.strip().split(", "), open(file).readlines())
    
    # Initialize variables
    if mapping is None:
        mapping = {}
    categorical_data = []
    numerical_data = [[],[]]
    target_data = []
    min_v = [999, 999]
    max_v = [-1, -1]
    
    # Map categorical features and save numerical fields
    for row in data:
        new_row = []
        for j, x in enumerate(row):
            if j == 0:
                numerical_data[1].append(x)
                if int(x) < min_v[1]:
                    min_v[1] = int(x)
                if int(x) > max_v[1]:
                    max_v[1] = int(x)
                continue
            if j == 7: 
                numerical_data[0].append(x)
                if int(x) < min_v[0]:
                    min_v[0] = int(x)
                if int(x) > max_v[0]:
                    max_v[0] = int(x)
                continue
            if j == 9:
                target_data.append(x)
                continue
            feature = (j, x)
            if feature not in mapping:
                mapping[feature] = len(mapping)
            new_row.append(mapping[feature])
        categorical_data.append(new_row)
        
    # Binarize categorical data
    print(str(len(categorical_data)) + ', ' + str(len(mapping)) + ', ' + str(len(numerical_data)))
          
    binary_data = np.zeros((len(categorical_data), len(mapping) + len(numerical_data)))
          
    for i, row in enumerate(categorical_data):
        for x in row:
            binary_data[i][x] = 1
            
    # Add weighted numerical data to binary data
    for i, nv in enumerate(numerical_data):
        div = (max_v[i] - min_v[i]) / 2
        for j, value in enumerate(nv):
            binary_data[j][binary_data[0].size - 1 - i] = int(value) / 50
    
    return [binary_data, mapping, target_data]
    

In [22]:
def euclidean(p1, p2):
    return np.linalg.norm(p1-p2)

In [23]:
def manhattan(p1, p2):
    return np.sum(abs(p1 - p2))

In [24]:
def k_NN(comp, arr, method, k):
    nn = []
    dist = []
    
    for j, data in enumerate(arr):
        
        # While nearest neighbor list does not have k elements
        if len(nn) < k:
            # If first element, add to nearest neighbor
            if len(nn) == 0:
                nn.append(j)
                dist.append(method(comp, arr[j]))
            
            # Else insert in ascending order to nearest neighbors
            else:
                for n in range(len(nn)):
                    if method(comp, arr[j]) < method(comp, arr[nn[n]]):
                        nn.insert(n, j)
                        dist.insert(n, method(comp, arr[j]))
                        break
                    elif n == len(nn) - 1:
                        nn.append(j)
                        dist.append(method(comp, arr[j]))
                        
        # Otherwise insert in ascending order and delete last element           
        else:
            for n in range(len(nn)):
                if method(comp, arr[j]) < method(comp, arr[nn[n]]):
                    nn.insert(n, j)
                    dist.insert(n, method(comp, arr[j]))
                    del nn[-1]
                    del dist[-1]
                    break
                    
    return [nn, dist]

In [25]:
def k_NN_predict(comp, bin_arr, method, k):
    
    # Map target possibilities
    mapping = {}
    for j, x in enumerate(bin_arr[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
    
    # Generate count for map from neighbors
    count = []
    for feature in mapping:
        count.append(0)
    
    nn_arr = k_NN(comp, bin_arr[0], method, k)
    
    for nn in nn_arr[0]:
        count[mapping[bin_arr[2][nn]]] += 1
    
    # Return prediction based on majority
    for feature in mapping:
        if count.index(max(count)) == mapping[feature]:
            return feature
    
    return

In [26]:
train = binarization('income.train.txt.5k')

5000, 90, 2


In [27]:
dev = binarization('income.dev.txt', train[1])

1000, 91, 2


In [28]:
train = binarization('income.train.txt.5k', dev[1])

5000, 91, 2


In [29]:
train[1] == dev[1]

True

In [30]:
len(dev[1])

91

In [31]:
len(train[1])

91

In [32]:
k_NN(dev[0][-1], train[0], euclidean, 5)

[[1010, 1713, 3769, 2003, 2450],
 [0.05999999999999983,
  0.16000000000000014,
  0.26,
  0.2828427124746192,
  0.3400000000000001]]

In [33]:
k_NN(dev[0][-1], train[0], manhattan, 5)

[[1010, 1713, 3769, 2450, 2003],
 [0.05999999999999983,
  0.16000000000000014,
  0.26,
  0.3400000000000001,
  0.40000000000000024]]

In [34]:
!tail -1  income.dev.txt 

'tail' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
!sed -n 1011p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [36]:
!sed -n 1714p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
!sed -n 3770p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
!sed -n 2004p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [39]:
!sed -n 3699p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [40]:
!sed -n 2451p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [41]:
k_NN(dev[0][-1], train[0], euclidean, 9)

[[1010, 1713, 3769, 2003, 2450, 3698, 3680, 681, 2731],
 [0.05999999999999983,
  0.16000000000000014,
  0.26,
  0.2828427124746192,
  0.3400000000000001,
  0.4004996878900156,
  0.4386342439892262,
  0.5599999999999999,
  1.4142135623730951]]

In [42]:
!sed -n 2451p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [43]:
!sed -n 3681p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
!sed -n 682p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
!sed -n 2732p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [46]:
k_NN(dev[0][-1], train[0], manhattan, 9)

[[1010, 1713, 3769, 2450, 2003, 3698, 681, 3680, 2731],
 [0.05999999999999983,
  0.16000000000000014,
  0.26,
  0.3400000000000001,
  0.40000000000000024,
  0.41999999999999993,
  0.5599999999999999,
  0.58,
  2.0]]

In [47]:
!sed -n 2004p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [48]:
!sed -n 3681p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [49]:
!sed -n 682p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [50]:
!sed -n 2732p income.train.txt.5k

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [51]:
k_NN_predict(dev[0][-1], train, euclidean, 5)

'<=50K'

In [52]:
def v_euclidean(arr, comp):
    return np.linalg.norm(arr - comp, axis=1)

In [53]:
def v_manhattan(arr,comp):
    return np.sum(abs(arr - comp), axis=1)

In [54]:
def v_k_NN(comp, arr, method, k):
    m_arr = method(arr, comp)
    nn_arr = np.argpartition(m_arr,k)[:k]
    dist = []
    
    for nn in nn_arr:
        dist.append(m_arr[nn])
        
    return [nn_arr, dist]

In [55]:
def v_k_NN_predict(comp, bin_arr, method, k, tar_map):
        
    # Map target possibilities
    mapping = tar_map
    
    
    # Generate count for map from neighbors
    count = []
    for feature in mapping:
        count.append(0)
    
    nn_arr = v_k_NN(comp, bin_arr[0], method, k)
    
    for nn in nn_arr[0]:
        count[mapping[bin_arr[2][nn]]] += 1
    
    # Return prediction based on majority
    for feature in mapping:
        if count.index(max(count)) == mapping[feature]:
            return [feature, mapping]

In [56]:
v_k_NN(dev[0][-1], train[0], v_euclidean, 5)

[array([1713, 3769, 2003, 1010, 2450], dtype=int64),
 [0.16000000000000014,
  0.26,
  0.2828427124746192,
  0.05999999999999983,
  0.3400000000000001]]

In [57]:
k_set = [1, 3, 5, 7, 9, 99, 999]

In [58]:
def error_rate(k_set, method, test_set):
    output = []
    mapping = {}
    
    for j, x in enumerate(train[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
                
    for k in k_set:
        print(k)
        correct = 0
        p_arr = None
        for i, person in enumerate(test_set[0]):
            p = v_k_NN_predict(person, train, method, k, mapping)
            
            # Build positive rate array
            if p_arr is None:
                p_arr = [p[1],[]]
                for e in p[1]:
                    p_arr[1].append(0)
                    
            # Update error and positive rate
            if p[0] == test_set[2][i]:
                correct += 1
            
            p_arr[1][p_arr[0][p[0]]] += 1
            
        print([correct , p_arr])
        output.append([correct / len(test_set[0]), p_arr])
        
    return output

In [59]:
s1 = time.time()

In [60]:
error_rate(k_set, v_euclidean, dev)

1
[767, [{'<=50K': 0, '>50K': 1}, [731, 269]]]
3
[808, [{'<=50K': 0, '>50K': 1}, [740, 260]]]
5
[823, [{'<=50K': 0, '>50K': 1}, [749, 251]]]
7
[835, [{'<=50K': 0, '>50K': 1}, [755, 245]]]
9
[840, [{'<=50K': 0, '>50K': 1}, [774, 226]]]
99
[842, [{'<=50K': 0, '>50K': 1}, [806, 194]]]
999
[821, [{'<=50K': 0, '>50K': 1}, [889, 111]]]


[[0.767, [{'<=50K': 0, '>50K': 1}, [731, 269]]],
 [0.808, [{'<=50K': 0, '>50K': 1}, [740, 260]]],
 [0.823, [{'<=50K': 0, '>50K': 1}, [749, 251]]],
 [0.835, [{'<=50K': 0, '>50K': 1}, [755, 245]]],
 [0.84, [{'<=50K': 0, '>50K': 1}, [774, 226]]],
 [0.842, [{'<=50K': 0, '>50K': 1}, [806, 194]]],
 [0.821, [{'<=50K': 0, '>50K': 1}, [889, 111]]]]

In [61]:
e1 = time.time()

In [62]:
e1 - s1

25.819942235946655

In [63]:
s2 = time.time()

In [64]:
error_rate(k_set, v_euclidean, train)

1
[4924, [{'<=50K': 0, '>50K': 1}, [3747, 1253]]]
3


KeyboardInterrupt: 

In [ ]:
e2 = time.time()

In [ ]:
e2 - s2

In [ ]:
s3 = time.time()

In [ ]:
error_rate(k_set, v_manhattan, dev)

In [ ]:
e3 = time.time()

In [ ]:
e3 - s3

In [ ]:
s4 = time.time()

In [ ]:
error_rate(k_set, v_manhattan, train)

In [ ]:
e4 = time.time()

In [ ]:
e4 - s4

In [ ]:
def binarize_all(file, mapping=None):
    # Intialize data array from file
    data = map(lambda s: s.strip().split(", "), open(file).readlines())
    
    # Initialize variables
    if mapping is None:
        mapping = {}
    categorical_data = []
    target_data = []
    min_v = [999, 999]
    max_v = [-1, -1]
    
    # Map categorical features and save numerical fields
    for row in data:
        new_row = []
        for j, x in enumerate(row):
            if j == 9:
                target_data.append(x)
                continue
            feature = (j, x)
            if feature not in mapping:
                mapping[feature] = len(mapping)
            new_row.append(mapping[feature])
        categorical_data.append(new_row)
        
    # Binarize categorical data
    print(str(len(categorical_data)) + ', ' + str(len(mapping)))
          
    binary_data = np.zeros((len(categorical_data), len(mapping)))
          
    for i, row in enumerate(categorical_data):
        for x in row:
            binary_data[i][x] = 1
    
    return [binary_data, mapping, target_data]

In [ ]:
def error_rate_all(k_set, method, test_set):
    output = []
    mapping = {}
    
    for j, x in enumerate(train[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
                
    for k in k_set:
        print(k)
        correct = 0
        p_arr = None
        for i, person in enumerate(test_set[0]):
            p = v_k_NN_predict(person, train_all, method, k, mapping)
            
            # Build positive rate array
            if p_arr is None:
                p_arr = [p[1],[]]
                for e in p[1]:
                    p_arr[1].append(0)
                    
            # Update error and positive rate
            if p[0] == test_set[2][i]:
                correct += 1
            
            p_arr[1][p_arr[0][p[0]]] += 1
            
        print([correct , p_arr])
        output.append([correct / len(test_set[0]), p_arr])
        
    return output

In [ ]:
train_all = binarize_all('income.train.txt.5k')

In [ ]:
dev_all = binarize_all('income.dev.txt', train_all[1])

In [ ]:
train_all = binarize_all('income.train.txt.5k', dev_all[1])

In [ ]:
s5 = time.time()

In [ ]:
error_rate_all(k_set, v_euclidean, dev_all)

In [ ]:
e5 = time.time()

In [ ]:
e5 - s5

In [ ]:
s6 = time.time()

In [ ]:
error_rate_all(k_set, v_euclidean, train_all)

In [ ]:
e6 = time.time()

In [ ]:
e6 - s6

In [ ]:
test = binarization('income.test.blind', train[1])

In [ ]:
def mass_predict(k_set, method, test_set):
    output = []
    
    mapping = {}
    
    for j, x in enumerate(train[2]):
            feature = x
            if feature not in mapping:
                mapping[feature] = len(mapping)
    
    for k in k_set:
        print(k)
        p_arr = None
        predictions = []
        for i, person in enumerate(test_set[0]):
            p = v_k_NN_predict(person, train, method, k, mapping)
            predictions.append(p)
            
            # Build positive rate array
            if p_arr is None:
                p_arr = [p[1],[]]
                for e in p[1]:
                    p_arr[1].append(0)

            # Update postive rate
            p_arr[1][p_arr[0][p[0]]] += 1
            
        print([p_arr])
        output.append(p_arr)
        output.append(predictions)
        
    return output

In [ ]:
s7 = time.time()

In [ ]:
blind = mass_predict([99], v_manhattan, test)

In [ ]:
e7 = time.time()

In [ ]:
e7 - s7

In [ ]:
with open('income.test.predicted', 'w') as out:
    for i, line in enumerate(open('income.test.blind').readlines()):
        out.write(line[:-2] + ', ' + blind[1][i][0] + '\n')

In [76]:
k_set = [50, 60, 70, 80, 90, 100]

In [77]:
s = time.time()

In [78]:
error_rate(k_set, v_euclidean, train)

50
[4102, [{'<=50K': 0, '>50K': 1}, [4039, 961]]]
60
[4100, [{'<=50K': 0, '>50K': 1}, [4029, 971]]]
70
[4108, [{'<=50K': 0, '>50K': 1}, [4031, 969]]]
80
[4102, [{'<=50K': 0, '>50K': 1}, [4039, 961]]]
90
[4108, [{'<=50K': 0, '>50K': 1}, [4039, 961]]]
100
[4107, [{'<=50K': 0, '>50K': 1}, [4046, 954]]]


[[0.8204, [{'<=50K': 0, '>50K': 1}, [4039, 961]]],
 [0.82, [{'<=50K': 0, '>50K': 1}, [4029, 971]]],
 [0.8216, [{'<=50K': 0, '>50K': 1}, [4031, 969]]],
 [0.8204, [{'<=50K': 0, '>50K': 1}, [4039, 961]]],
 [0.8216, [{'<=50K': 0, '>50K': 1}, [4039, 961]]],
 [0.8214, [{'<=50K': 0, '>50K': 1}, [4046, 954]]]]

In [79]:
e = time.time()

In [80]:
e - s

69.71623373031616